# Azure AI Agents with Model Context Protocol (MCP) Support - Python

यो नोटबुकले Azure AI Agents लाई Model Context Protocol (MCP) उपकरणहरूसँग Python मा कसरी प्रयोग गर्ने भनेर देखाउँछ। यसले देखाउँछ कि कसरी एक बुद्धिमान एजेन्ट सिर्जना गर्न सकिन्छ जसले बाह्य MCP सर्भरहरू (जस्तै Microsoft Learn) लाई कुञ्जीबिनाको प्रमाणीकरण प्रयोग गरेर उन्नत क्षमताहरूको लागि प्रयोग गर्न सक्छ।


## आवश्यक Python प्याकेजहरू स्थापना गर्नुहोस्

पहिले, हामीले आवश्यक Python प्याकेजहरू स्थापना गर्नुपर्छ:
- **azure-ai-projects**: कोर Azure AI Projects SDK
- **azure-ai-agents**: एजेन्टहरू सिर्जना र व्यवस्थापन गर्नको लागि Azure AI Agents SDK
- **azure-identity**: DefaultAzureCredential प्रयोग गरेर कुञ्जीबिनाको प्रमाणीकरण प्रदान गर्दछ
- **mcp**: Python को लागि Model Context Protocol कार्यान्वयन


## कीलेस प्रमाणीकरणका फाइदाहरू

यो नोटबुकले **कीलेस प्रमाणीकरण** प्रदर्शन गर्दछ, जसले धेरै फाइदाहरू प्रदान गर्दछ:
- ✅ **एपीआई कुञ्जीहरू व्यवस्थापन गर्न आवश्यक छैन** - Azure पहिचान-आधारित प्रमाणीकरण प्रयोग गर्दछ
- ✅ **सुरक्षा सुधारिएको छ** - कोड वा कन्फिगरेसन फाइलहरूमा कुनै गोप्य जानकारी भण्डारण हुँदैन
- ✅ **स्वचालित प्रमाण-पत्र परिक्रमा** - Azure ले प्रमाण-पत्र जीवनचक्र व्यवस्थापन गर्छ
- ✅ **भूमिका-आधारित पहुँच नियन्त्रण** - Azure RBAC प्रयोग गरेर सूक्ष्म अनुमति व्यवस्थापन
- ✅ **बहु-पर्यावरण समर्थन** - विकास र उत्पादनमा सहज रूपमा काम गर्दछ

`DefaultAzureCredential` ले उपलब्ध सबैभन्दा उपयुक्त प्रमाण-पत्र स्रोत स्वचालित रूपमा चयन गर्दछ:
1. **प्रबन्धित पहिचान** (Azure मा चल्दा)
2. **Azure CLI** प्रमाण-पत्रहरू (स्थानीय विकासको समयमा)
3. **भिजुअल स्टुडियो** प्रमाण-पत्रहरू
4. **पर्यावरण भेरिएबलहरू** (यदि कन्फिगर गरिएको छ भने)
5. **इन्टरएक्टिभ ब्राउजर** प्रमाणीकरण (बैकअपको रूपमा)


## कीलेस प्रमाणीकरण सेटअप

**कीलेस प्रमाणीकरणको लागि आवश्यकताहरू:**

### स्थानीय विकासको लागि:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Azure वातावरणहरूको लागि:
- आफ्नो Azure स्रोतमा **System-assigned Managed Identity** सक्षम गर्नुहोस्
- व्यवस्थापन पहिचानलाई उपयुक्त **RBAC भूमिका**हरू प्रदान गर्नुहोस्:
  - Azure OpenAI पहुँचको लागि `Cognitive Services OpenAI User`
  - Azure AI Projects पहुँचको लागि `AI Developer`

### वातावरण चरहरू (वैकल्पिक):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**कुनै API कुञ्जीहरू वा जडान स्ट्रिङहरू आवश्यक छैन!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## आवश्यक पुस्तकालयहरू आयात गर्नुहोस्

आवश्यक Python मोड्युलहरू आयात गर्नुहोस्:
- **os, time**: वातावरण भेरिएबलहरू र ढिलाइहरूको लागि मानक Python पुस्तकालयहरू
- **AIProjectClient**: Azure AI Projects को मुख्य क्लाइन्ट
- **DefaultAzureCredential**: Azure सेवाहरूको लागि कुञ्जीबिनाको प्रमाणीकरण
- **MCP-सँग सम्बन्धित कक्षाहरू**: MCP उपकरणहरू सिर्जना र व्यवस्थापन गर्न र स्वीकृतिहरू ह्यान्डल गर्न


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## MCP सर्भर सेटिङहरू कन्फिगर गर्नुहोस्

पर्यावरण भेरिएबलहरू प्रयोग गरेर MCP सर्भर कन्फिगरेसन सेट गर्नुहोस्, जसमा डिफल्ट मानहरू पनि छन्:
- **MCP_SERVER_URL**: MCP सर्भरको URL (डिफल्ट Microsoft Learn API हो)
- **MCP_SERVER_LABEL**: MCP सर्भरलाई चिन्हित गर्नको लागि लेबल (डिफल्ट "mslearn" हो)

यो विधिले विभिन्न वातावरणहरूमा लचिलो कन्फिगरेसनको अनुमति दिन्छ।


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Azure AI प्रोजेक्ट क्लाइन्ट (कुञ्जीबिनाको प्रमाणीकरण) सिर्जना गर्नुहोस्

**कुञ्जीबिनाको प्रमाणीकरण** प्रयोग गरेर Azure AI प्रोजेक्ट क्लाइन्ट आरम्भ गर्नुहोस्:
- **endpoint**: Azure AI Foundry प्रोजेक्टको endpoint URL
- **credential**: सुरक्षित, कुञ्जीबिनाको प्रमाणीकरणको लागि `DefaultAzureCredential()` प्रयोग गर्दछ
- **API कुञ्जीहरू आवश्यक छैनन्**: उपलब्ध सबैभन्दा राम्रो credential स्वचालित रूपमा पत्ता लगाउँछ र प्रयोग गर्दछ

**प्रमाणीकरण प्रक्रिया:**
1. Azure वातावरणमा Managed Identity जाँच गर्दछ
2. स्थानीय विकासको लागि Azure CLI credentials प्रयोग गर्दछ
3. आवश्यक परे अन्य उपलब्ध credential स्रोतहरू प्रयोग गर्दछ

यस विधिले तपाईंको कोडमा API कुञ्जीहरू वा connection strings व्यवस्थापन गर्नुपर्ने आवश्यकता हटाउँछ।


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## MCP उपकरण परिभाषा बनाउनुहोस्

Microsoft Learn MCP सर्भरमा जडान गर्न MCP उपकरण बनाउनुहोस्:
- **server_label**: MCP सर्भरको पहिचानकर्ता
- **server_url**: MCP सर्भरको URL अन्तिम बिन्दु
- **allowed_tools**: वैकल्पिक सूची जसले कुन उपकरणहरू प्रयोग गर्न सकिन्छ भनेर सीमित गर्दछ (खाली सूचीले सबै उपकरणहरूलाई अनुमति दिन्छ)

यो उपकरणले एजेन्टलाई Microsoft Learn दस्तावेजहरू र स्रोतहरू पहुँच गर्न सक्षम बनाउनेछ।


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## एजेन्ट बनाउनुहोस् र संवाद सञ्चालन गर्नुहोस् (कुञ्जीबिहीन कार्यप्रवाह)

यो विस्तृत खण्डले **कुञ्जीबिहीन एजेन्ट कार्यप्रवाह** को सम्पूर्ण प्रक्रिया देखाउँछ:

1. **एआई एजेन्ट बनाउनुहोस्**: GPT-4.1 नानो मोडेल र MCP उपकरणहरूसँग एजेन्ट सेटअप गर्नुहोस्  
2. **थ्रेड बनाउनुहोस्**: संवादका लागि थ्रेड स्थापना गर्नुहोस्  
3. **सन्देश पठाउनुहोस्**: एजेन्टलाई Azure OpenAI र OpenAI बीचको भिन्नताबारे सोध्नुहोस्  
4. **उपकरण स्वीकृति व्यवस्थापन गर्नुहोस्**: आवश्यक परेमा MCP उपकरण कलहरू स्वचालित रूपमा स्वीकृत गर्नुहोस्  
5. **कार्य सञ्चालनको निगरानी गर्नुहोस्**: एजेन्टको प्रगति ट्र्याक गर्नुहोस् र आवश्यक कार्यहरू व्यवस्थापन गर्नुहोस्  
6. **परिणाम देखाउनुहोस्**: संवाद र उपकरण प्रयोगको विवरण प्रदर्शन गर्नुहोस्  

**कुञ्जीबिहीन विशेषताहरू:**
- ✅ **कुनै हार्डकोड गरिएको गोप्य जानकारी छैन** - सबै प्रमाणीकरण Azure पहिचानद्वारा व्यवस्थापन गरिन्छ  
- ✅ **डिफल्ट रूपमा सुरक्षित** - भूमिका-आधारित पहुँच नियन्त्रण प्रयोग गर्दछ  
- ✅ **सरल परिनियोजन** - प्रमाणपत्र व्यवस्थापन आवश्यक छैन  
- ✅ **अडिट-अनुकूल** - सबै पहुँच Azure पहिचानद्वारा ट्र्याक गरिन्छ  

एजेन्टले MCP उपकरणहरू प्रयोग गरेर Microsoft Learn स्रोतहरूमा पूर्ण सुरक्षा र कुनै API कुञ्जी व्यवस्थापन बिना पहुँच गर्नेछ।  


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरेर अनुवाद गरिएको छ। हामी शुद्धताको लागि प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादहरूमा त्रुटि वा अशुद्धता हुन सक्छ। यसको मूल भाषा मा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्वपूर्ण जानकारीको लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याको लागि हामी जिम्मेवार हुने छैनौं।
